# 영화 리뷰 감정 분석
**Recurrent Neural Network (RNN) 을 이용해 IMDB 데이터를 가지고 텍스트 감정분석을 해 봅시다.**

순차적인 데이터를 가지고 학습한다는 점에서 RNN은 '자연어 처리'(Natural Language Processing) 분야에 매우 큰 부각을 나타내 왔습니다.
이번 프로젝트를 통해 가장 기본적인 자연어 처리작업이라고 할 수 있는 '텍스트 감정분석'(Text Sentiment Analysis)을 RNN 을 이용해 구현하고 공부해 보겠습니다.

이번 책에서 처음으로 접하는 텍스트 형태의 데이터셋인 IMDB 데이터셋은 50,000건의 영화 리뷰로 이루어져 있습니다.
각 리뷰는 다수의 영어 문장들로 이루어져 있으며, 평점이 7점 이상의 긍정적인 영화 리뷰는 2로, 평점이 4점 이하인 부정적인 영화 리뷰는 1로 레이블링 되어 있습니다. 영화 리뷰 데이터를 RNN 에 입력시켜 리뷰의 전체 내용을 이해하여 압축하고, 이렇게 압축된 리뷰를 기반으로 영화 리뷰가 긍정적인지 부정적인지 판단해주는 간단한 분류 모델을 구현하는 것이 이번 프로젝트의 목표입니다.

가장 먼저 모델 구현과 학습에 필요한 라이브러리 들을 임포트 해 줍니다. 여기서 처음 등장하는 Torch Text 는 자연어 처리 딥러닝에 쓰이는 다양한 데이터셋들을 포함하고 있으며, 데이터셋을 학습에 편한 형태로 바꿔주는 기능을 해 주는 편리한 오픈소스입니다.

In [3]:
import os
import sys
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

모델 구현과 학습에 필요한 하이퍼파라미터 들을 정의해 줍니다.

In [4]:
# get hyper parameters
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 40
torch.manual_seed(42)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## RNN 기반 모델 클래스 생성하기

BasicLSTM 라고 하는 RNN 을 포함하는 신경망 모델을 만들어 보겠습니다. 여타 다른 신경망 모델과 같이 파이토치의 nn.Module 을 상속받습니다.

```python
class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
```

### RNN 에 필요한 파라미터 함수 정의하기

#### 워드 임베딩

가장 먼저 해야할 일은 영화 리뷰 속 단어들을 텐서 형태로 바꾸어 주는 일입니다.
이러한 과정을 바로 바로 워드 임베딩(Word Embedding)이라고 합니다.
사실 워드 임베딩을 하기 위해서는 텍스트 데이터에 여러 전처리 과정을 거쳐야 합니다.
다행히도 파이토치의 nn.Embedding 이라는 함수를 사용하면 직접 우리가 이런 복잡한 데이터 처리를 할 필요는 없습니다.

nn.Embedding 함수는 2개의 파라미터를 입력받습니다. 이 중 첫번째는 전체 데이터셋 속 모든 단어를 사전 형태로 나타냈을 때
이 사전속 단어의 갯수라고 할 수 있는 n_vocab이라는 숫자입니다.
두번째 파라미터는 embed 라는 숫자입니다. 이 숫자는 쉽게 말해 임베딩된 단어 텐서가 지니는 차원값 이라고 할 수 있습니다.
즉, 한 영화 리뷰속 모든 단어가 임베딩을 거치면 영화 리뷰는 embed 만큼 특성값을 지닌 단어 텐서들이 차례대로 나열된 배열 형태로 나타내어 집니다.

```python
class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
```

다음으로 drop out 을 정의해 주고 본격적으로 RNN 모델을 정의합니다. 원시적인 형태의 RNN 보다는 Gradient Vanishing 과
Gradient Explosion 이라는 RNN 의 고질적인 문제에서 좀 더 자유로운 LSTM(Long Short Term Memory) 라는 RNN 을 사용하겠습니다.

```python
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim,
                            num_layers=self.n_layers,
                            dropout=dropout_p,
                            batch_first=True)
```

앞에서 설명했듯, RNN 은 텐서의 배열을 하나의 텐서로 압축시킵니다.
하지만 사실상 RNN의 기능은 여기서 끝나기 때문에 모델이 영화 리뷰가 긍정적인지, 부정적인지 말해주는 결과값을 출력하려면 RNN 만으로는 부족합니다.
즉, 모델이 결과값을 내려면 RNN 을 통해 압축된 텐서를 다음과 같이 다층신경망에 입력시켜야 합니다.

```python
        self.out = nn.Linear(self.hidden_dim, n_classes)
```


### Forward 함수 정의하기

본격적으로 모델에 입력된 텍스트 데이터가 어떤 전처리 과정을 거치고 신경망에 입력되는지 정의하는 forward 함수를 구현합니다.
모델에 입력되는 데이터 x 는 한 batch 속에 있는 모든 영화평 입니다.
이들이 embed 함수를 통해 워드 임베딩을 하게 되면 LSTM 에 입력될 수 있는 형태가 됩니다.

```python
    def forward(self, x):
        x = self.embed(x)  #  [b, i] -> [b, i, e]
```

#### Initial Hidden State 정의하기
보통의 신겨망이라면 이제 바로 신경망 모듈의 forward 함수를 호출해도 되겠지만 
LSTM 과 같은 RNN 계열의 신경망은 입력 데이터 말고도 밑의 코드처럼 hidden state 이라는 텐서를 정의하고 신경망에 입력해 줘야 합니다.

```python
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, h_0)  # [i, b, h]
```

hidden state 인 h_0 를 정의하는 _init_state 함수는 신경망 객체 모델 속에서 다음과 같이 정의됩니다.

```python
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return (
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        )
```

next(self.parameters()).data 를 통해 모델 속 가중치 텐서를 weight 이라는 변수에 대입시킵니다.
그리고 new() 함수를 이용해 weight 텐서와 같은 자료형을 갖고 있지만 (n_layers, batch_size, hidden_dim) 모양을 가진 텐서 두개를 정의합니다. 그리고 이 두 텐서에 zero_() 함수를 호출함으로써 텐서 속 모든 원소값을 0 으로 바꿔줍니다. 대부분의 RNN 계열의 신경망은 hidden state 로써 하나의 텐서만을 요구하지만 좀 더 복잡한 구조를 가진 LSTM 은 이렇게 같은 모양의 텐서 두 개를 정의해 줘야 합니다.

hidden state 와 영화 리뷰 데이터 x 를 입력받은 LSTM 은 영화 리뷰의 길이 만큼의 hidden_state 텐서들을 리턴합니다. 이들 중 마지막 텐서가 바로
영화 리뷰를 압축한 텐서 입니다. 이 텐서를 h_t 라고 정의하겠습니다.

```python
        h_t = x[:,-1,:]
```

이제 영화 리뷰 속 내용을 압축한 h_t 텐서를 다층신경망에 입력시켜 결과를 출력해야 합니다.

```python
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
```



## RNN 기반 모델 클래스 코드

In [5]:
class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim,
                            num_layers=self.n_layers,
                            dropout=dropout_p,
                            batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)  #  [b, i] -> [b, i, e]
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, h_0)  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return (
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        )

### train 함수와 evaluate 함수 구현

전 장들과 마찬가지로 train 과 evaluate 함수를 구현합니다.

In [6]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # index align
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        if b % 100 == 0:
            corrects = (logit.max(1)[1].view(y.size()).data == y.data).sum()
            accuracy = 100.0 * corrects / batch.batch_size
            sys.stdout.write(
                '\rBatch[%d] - loss: %.6f  acc: %.2f' %
                (b, loss.item(), accuracy))

In [7]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, avg_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # index align
        logit = model(x)
        loss = F.cross_entropy(logit, y, size_average=False)
        avg_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = avg_loss / size
    accuracy = 100.0 * corrects / size
    return avg_loss, accuracy

# IMDB 데이터셋 가져오기

모델 구현과 신경망 학습에 필요한 함수를 구현했으면 본격적으로 IMDB 데이터셋을 가져와 보겠습니다.
사실 아무 가공처리를 가하지 않은 텍스트 형태의 데이터셋을 신경망에 입력하는데까지는 매우 번거로운 작업을 필요로합니다.
그러므로 우리는 이러한 전처리 작업들을 대신 해주는 Torch Text 라는 유용한 라이브러리를 사용해 IMDB 데이터셋을 가져오겠습니다.

가장 먼저 텍스트 형태의 영화 리뷰들과 그에 해당하는 레이블을 텐서로 바꿔줄 때 필요한 설정사항들을 정해줘야 합니다.
그러기 위해 이러한 설정정보를 담고있는 TEXT 와 LABEL 이라는 객체를 생성합니다. 

```python
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

```

sequential 이라는 파라미터를 이용해 데이터셋이 순차적 데이터셋이라고 명시해 주고 batch_first 파라미터로 신경망에 입력되는 텐서의 첫번째 차원값이 batch_size 가 되도록 정해줍니다.
마지막으로 lower 변수를 이용해 텍스트 데이터 속 모든 영문 알파벳이 소문자가 되도록 설정해 줍니다.

그 다음으로는 datasets 객체의 splits 함수를 이용해 모델에 입력되는 데이터셋을 만들어줍니다.

```python
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
```

이제 만들어진 데이터셋을 이용해 전에 설명한 워드 임베딩에 필요한 워드 사전(Word Vocabulary)를 만들어줍니다.

```python
TEXT.build_vocab(train_data, min_freq=5)
LABEL.build_vocab(train_data)
```

min_freq 은 학습데이터 속에서 최소한 5번 이상 등장한 단어들만을 사전속에 정의하겠다는 뜻입니다. 즉 학습 데이터 속에서 드물게 출현하는 단어는 'unk'(Unknown) 이라는 토큰으로 정의됩니다.

그 다음으로는 train_data 와 test_data 에서 batch tensor 를 generate 할 수 있는 iterator 를 만들어 줍니다.

```python
train_iter, test_iter = data.BucketIterator.splits(
        (train_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)
```

마지막으로 사전 속 단어들의 숫자와 레이블의 수를 정해주는 변수를 만들어 줍니다.

```python
vocab_size = len(TEXT.vocab)
n_classes = 2
```

In [8]:
# load data
print("\nLoading data...")
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
TEXT.build_vocab(train_data, min_freq=5)
LABEL.build_vocab(train_data)

train_iter, test_iter = data.BucketIterator.splits(
        (train_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 2
#len(LABEL.vocab) - 1


Loading data...


In [9]:
print("[TRAIN]: %d \t [TEST]: %d \t [VOCAB] %d \t [CLASSES] %d"
      % (len(train_iter),len(test_iter), vocab_size, n_classes))

[TRAIN]: 391 	 [TEST]: 391 	 [VOCAB] 46159 	 [CLASSES] 2


이제 학습에 필요한 모든것이 준비됬습니다. 
RNN 기반의 신경망 모델을 생성하고 학습을 시작합니다.

In [ ]:
model = BasicLSTM(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print(model)

Building Basic LSTM model...
BasicLSTM(
  (embed): Embedding(46159, 128)
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(128, 256, batch_first=True, dropout=0.5)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


/home/keon/.local/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, test_iter)

    print("\n[Epoch: %d] val_loss:%5.2f | acc:%5.2f" % (e, val_loss, val_accuracy))
    
    # Save the model if the validation loss is the best we've seen so far.
#     if not best_val_loss or val_loss < best_val_loss:
#         if not os.path.isdir("snapshot"):
#             os.makedirs("snapshot")
#         torch.save(model.state_dict(), './snapshot/convcnn.pt')
#         best_val_loss = val_loss

In [ ]:
# class BasicRNN(nn.Module):
#     """
#         Basic RNN
#     """
#     def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
#         super(BasicRNN, self).__init__()
#         print("Building Basic RNN model...")
#         self.n_layers = n_layers
#         self.hidden_dim = hidden_dim

#         self.embed = nn.Embedding(n_vocab, embed_dim)
#         self.dropout = nn.Dropout(dropout_p)
#         self.rnn = nn.RNN(embed_dim, hidden_dim, n_layers,
#                           dropout=dropout_p, batch_first=True)
#         self.out = nn.Linear(self.hidden_dim, n_classes)

#     def forward(self, x):
#         embedded = self.embed(x)  #  [b, i] -> [b, i, e]
#         _, hidden = self.rnn(embedded)
#         self.dropout(hidden)
#         hidden = hidden.squeeze()
#         logit = self.out(hidden)  # [b, h] -> [b, o]
#         return logit